# TASK 2.2: Tracking with a Kalman Filter

In [2]:
from __future__ import print_function
from skimage import io
from sort import Sort
from IPython import display as dp
from pqdm.processes import pqdm
import matplotlib.pyplot as plt  
import matplotlib.patches as patches
import numpy as np
import pandas as pd
import pickle as pkl
import os
import time
import cv2
%matplotlib inline

First we load all the data

In [3]:
# Detection to DataFrame
def get_detection_dataframe(detections):
    bboxes = []
    bsizes = []
    bdetections = []
    tracks = list(range(0, len(detections)))
    
    colours = []
    for i in range(len(detections)):
        colours.append(tuple(np.random.choice(range(256), size=3).astype('int')))
    
    for detection in detections:
        bbox = np.array(detection.getBBox()).astype('int')
        bboxes.append(bbox)

        bsize = int(detection.areaOfRec())
        bsizes.append(bsize)

        bdetections.append(detection)

    detec = {
        'track': tracks,
        'detection': bdetections,
        'bbox': bboxes,
        'size': bsizes,
        'colour': colours,
    }
    detections_pd = pd.DataFrame(detec)
    detections_pd = detections_pd.sort_values(by=['size'], ascending=False)
    detections_pd = detections_pd.reset_index(drop=True)
    
    return detections_pd

In [4]:
# Load paths
file_path = 'detection_pkls/retinanet_101_detections.pkl'
data_path = '../datasets/AICity_data/train/S03/c010/'
video_path = '../datasets/AICity_data/train/S03/c010/vdo.avi'

# Load detections
with open(file_path , 'rb') as f:
    all_detections = pkl.load(f)

# Load video frames
vidcap = cv2.VideoCapture(video_path)
num_frames = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))

In [5]:
all_detections_df = pd.concat(pqdm([all_detections[i] for i in all_detections.keys()],get_detection_dataframe, n_jobs=2))
all_detections_df[all_detections_df["track"] == 0]

QUEUEING TASKS | : 100%|██████████| 2141/2141 [00:00<00:00, 11812.87it/s]
PROCESSING TASKS | : 100%|██████████| 2141/2141 [00:03<00:00, 651.64it/s]
COLLECTING RESULTS | : 100%|██████████| 2141/2141 [00:00<00:00, 645579.07it/s]


,track,detection,bbox,size,colour
0,0,"Frame 0, TL [1284.8372802734375,362.1326904296...","[1284, 362, 1520, 539]",41907,"(175, 103, 205)"
0,0,"Frame 1, TL [1285.0892333984375,362.0698242187...","[1285, 362, 1520, 540]",42008,"(175, 103, 205)"
0,0,"Frame 2, TL [1285.4013671875,362.3092346191406...","[1285, 362, 1520, 540]",41872,"(99, 112, 69)"
0,0,"Frame 3, TL [1285.2103271484375,362.5265502929...","[1285, 362, 1521, 540]",42016,"(226, 156, 180)"
0,0,"Frame 4, TL [1285.2891845703125,362.7458190917...","[1285, 362, 1520, 540]",41972,"(99, 112, 69)"
...,...,...,...,...,...
0,0,"Frame 2136, TL [1284.1539306640625,361.2304992...","[1284, 361, 1519, 540]",42117,"(9, 1, 34)"
0,0,"Frame 2137, TL [1284.204345703125,361.03491210...","[1284, 361, 1519, 540]",42141,"(58, 61, 154)"
0,0,"Frame 2138, TL [1283.7984619140625,361.0021972...","[1283, 361, 1519, 540]",42153,"(123, 175, 139)"
0,0,"Frame 2139, TL [1284.388916015625,360.54635620...","[1284, 360, 1519, 540]",42145,"(58, 102, 80)"


Starting Tracking

In [ ]:
"""
display = True
total_time = 0.0
total_frames = 0
out = []

if display:
    plt.ion() # for iterative display
    fig, ax = plt.subplots(1, 2,figsize=(20,20))
"""